# Borrar las columnas del CSV que no vamos a utilizar como la de Generation y la de Legendary

In [1]:
import pandas as pd

# Cargar el CSV
df = pd.read_csv('pokemon.csv')

# Eliminar las columnas 'Type 1' y 'HP'
df = df.drop(columns=['Generation', 'Legendary'])

# Guardar el CSV actualizado
df.to_csv('pokemon_actualizado.csv', index=False)

print("Las columnas han sido eliminadas y el archivo ha sido guardado como 'archivo_actualizado.csv'.")

Las columnas han sido eliminadas y el archivo ha sido guardado como 'archivo_actualizado.csv'.


In [2]:
import pandas as pd
import requests

# Cargar el CSV existente (sin las columnas 'Generation' y 'Legendary')
df = pd.read_csv('pokemon_actualizado.csv')

# Función para obtener el detalle de un Pokémon desde la API
def obtener_pokemon(id_pokemon):
    url = f'https://pokeapi.co/api/v2/pokemon/{id_pokemon}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return {
            'id': data['id'],
            'Name': data['name'].capitalize(),
            'Type 1': data['types'][0]['type']['name'].capitalize(),
            'Type 2': data['types'][1]['type']['name'].capitalize() if len(data['types']) > 1 else '',
            'Total': sum(stat['base_stat'] for stat in data['stats']),
            'HP': data['stats'][0]['base_stat'],
            'Attack': data['stats'][1]['base_stat'],
            'Defense': data['stats'][2]['base_stat'],
            'Sp. Atk': data['stats'][3]['base_stat'],
            'Sp. Def': data['stats'][4]['base_stat'],
            'Speed': data['stats'][5]['base_stat'],
        }
    else:
        return None

# Recorremos los Pokémon desde el ID 722 hasta el 1025
for id_pokemon in range(722, 1026):
    pokemon_data = obtener_pokemon(id_pokemon)
    if pokemon_data:
        # Crear un DataFrame temporal para agregarlo
        pokemon_df = pd.DataFrame([pokemon_data])
        # Concatenar el nuevo DataFrame al DataFrame principal
        df = pd.concat([df, pokemon_df], ignore_index=True)

# Guardar el DataFrame actualizado
df.to_csv('pokemon_completo.csv', index=False)

print("El dataset ha sido actualizado con los nuevos Pokémon.")

El dataset ha sido actualizado con los nuevos Pokémon.


In [1]:
import pandas as pd
import uuid
from cassandra.cluster import Cluster

# Conectar a Cassandra
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

# Seleccionar Keyspace
session.set_keyspace('poke_builder')

# Cargar dataset
df = pd.read_csv("pokemon_completo.csv")

# Insertar datos en Cassandra
for _, row in df.iterrows():
    session.execute("""
        INSERT INTO pokemon (id, name, type1, type2, total, hp, attack, defense, sp_attack, sp_defense, speed)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        uuid.uuid4(),
        row['Name'],
        row['Type 1'],
        None if pd.isna(row['Type 2']) else row['Type 2'],  # Evitar NaN en Type 2
        row['Total'],
        row['HP'],
        row['Attack'],
        row['Defense'],
        row['Sp. Atk'],
        row['Sp. Def'],
        row['Speed']
    ))

print("Datos insertados correctamente ✅")


Datos insertados correctamente ✅


In [2]:
# Matriz de efectividad de tipos en Pokemon
type_chart = {
    'Acero': {'Acero': 0.5, 'Agua': 0.5, 'Eléctrico': 0.5, 'Fuego': 0.5, 'Hada': 2, 'Hielo': 2, 'Roca': 2},
    'Agua': {'Fuego': 2, 'Agua': 0.5, 'Planta': 0.5, 'Tierra': 2, 'Roca': 2, 'Dragón': 0.5},
    'Bicho': {'Acero': 0.5, 'Fantasma': 0.5, 'Fuego': 0.5, 'Hada': 0.5, 'Lucha': 0.5, 'Planta': 2, 'Psíquico': 2, 'Siniestro': 2, 'Veneno': 0.5},
    'Dragón': {'Acero': 0.5, 'Dragón': 2, 'Hada': 0},
    'Eléctrico': {'Agua': 2, 'Eléctrico': 0.5, 'Planta': 0.5, 'Tierra': 0, 'Volador': 2, 'Dragón': 0.5},
    'Fantasma': {'Fantasma': 2, 'Normal': 0, 'Psíquico': 2, 'Siniestro': 0.5},
    'Fuego': {'Fuego': 0.5, 'Agua': 0.5, 'Planta': 2, 'Hielo': 2, 'Bicho': 2, 'Roca': 0.5, 'Dragón': 0.5, 'Acero': 2},
    'Hada': {'Acero': 0.5, 'Dragón': 2, 'Fuego': 0.5, 'Lucha': 2, 'Siniestro': 2, 'Veneno': 0.5},
    'Hielo': {'Fuego': 0.5, 'Agua': 0.5, 'Planta': 2, 'Hielo': 0.5, 'Tierra': 2, 'Volador': 2, 'Dragón': 2, 'Acero': 0.5},
    'Lucha': {'Acero': 2, 'Bicho': 0.5, 'Fantasma': 0, 'Hada': 0.5, 'Hielo': 2,'Normal': 2, 'Psíquico': 0.5, 'Roca': 2, 'Siniestro': 2, 'Veneno': 0.5},
    'Normal': {'Roca': 0.5, 'Fantasma': 0, 'Acero': 0.5},
    'Planta': {'Fuego': 0.5, 'Agua': 2, 'Planta': 0.5, 'Veneno': 0.5, 'Tierra': 2, 'Volador': 0.5, 'Bicho': 0.5, 'Roca': 2, 'Dragón': 0.5, 'Acero': 0.5},
    'Psíquico': {'Acero': 0.5, 'Lucha': 2, 'Psíquico': 0.5, 'Siniestro': 0, 'Veneno': 2},
    'Roca': {'Acero': 0.5, 'Bicho': 2, 'Fuego': 2, 'Hielo': 2, 'Lucha': 0.5, 'Tierra': 0.5, 'Volador': 2},
    'Siniestro': {'Fantasma': 2, 'Hada': 0.5, 'Lucha': 0.5, 'Psíquico': 2, 'Siniestro': 0.5},
    'Tierra': {'Acero': 2, 'Bicho': 0.5, 'Eléctrico': 2, 'Fuego': 2, 'Planta': 0.5, 'Roca': 2, 'Veneno': 2, 'Volador': 0},
    'Veneno': {'Acero': 0, 'Hada': 2, 'Planta': 2, 'Roca': 0.5, 'Fantasma': 0.5, 'Veneno': 0.5, 'Tierra': 0.5},
    'Volador': {'Acero': 0.5, 'Bicho': 2, 'Eléctrico': 0.5, 'Lucha': 2, 'Planta': 2, 'Roca': 0.5}
}

In [3]:
# Función para calcular la efectividad ofensiva con manejo de errores
def calcular_ofensiva(tipo_atacante, tipos_defensor):
    """Calcula la efectividad ofensiva del atacante contra el defensor."""
    multiplicador = 1
    if tipo_atacante not in type_chart:
        return multiplicador  # Retorna 1 si el tipo atacante no está en la tabla

    for tipo_defensor in tipos_defensor:
        multiplicador *= type_chart[tipo_atacante].get(tipo_defensor, 1)  # Si no hay valor, asume 1

    return multiplicador

# Función para calcular la efectividad defensiva
def calcular_defensiva(tipos_defensor, tipo_atacante):
    """Calcula la resistencia del defensor contra el atacante."""
    return calcular_ofensiva(tipo_atacante, tipos_defensor)

# Función mejorada para evaluar el combate
def evaluar_combate(tipos_A, tipos_B):
    """Evalúa la ventaja de A sobre B considerando ataque y defensa."""
    if not tipos_A or not tipos_B:  # Evita divisiones por cero
        return 1

    ofensiva_A_vs_B = sum(calcular_ofensiva(tipo, tipos_B) for tipo in tipos_A) / max(len(tipos_A), 1)
    defensiva_A_vs_B = sum(calcular_defensiva(tipos_A, tipo) for tipo in tipos_B) / max(len(tipos_B), 1)

    return ofensiva_A_vs_B / defensiva_A_vs_B  # A mayor valor, mejor A contra B

In [4]:
def recomendar_equipo(equipo_rival):
    session = cluster.connect('poke_builder')

    # Obtener todos los Pokémon
    rows = session.execute("SELECT * FROM pokemon")
    mejores_pokemon = []

    for pokemon in rows:
        ventaja = sum(evaluar_combate([pokemon.type1, pokemon.type2], rival) for rival in equipo_rival)
        mejores_pokemon.append((pokemon.name, ventaja))

    # Ordenar por mayor ventaja
    mejores_pokemon.sort(key=lambda x: x[1], reverse=True)

    return [p[0] for p in mejores_pokemon[:6]]  # Devuelve los mejores 6

In [5]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recomendar', methods=['POST'])
def recomendar():
    equipo_rival = request.json['equipo']
    equipo_recomendado = recomendar_equipo(equipo_rival)
    return jsonify({"equipo_recomendado": equipo_recomendado})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\INES\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
